# Monitor

A notebook to monitor the status of bulk asset uploads (with [data_loading.py](data_loading.py))

In [122]:
from pathlib import Path

PATH_TO_EXAMPLE_OBJECTS = "/home/yunusskeete/Documents/data/3D/Cap3D/local-split/unzips/compressed_imgs_perobj_00.zip/Cap3D_Objaverse_renderimgs"
path_to_example_objects = Path(PATH_TO_EXAMPLE_OBJECTS)
len(list(path_to_example_objects.iterdir()))

15000

In [200]:
import requests
import json
import weaviate
import weaviate.classes as wvc
from weaviate.classes.init import AdditionalConfig, Timeout
import weaviate.classes.query as wq
from weaviate.connect import ConnectionParams

HTTP_HOST = "localhost"
HTTP_PORT = 8080
HTTP_SECURE = False

GRPC_HOST = "localhost"
GRPC_PORT = 50051
GRPC_SECURE = False

INTIALISATION_TIMEOUT_S = 2
QUERY_TIMEOUT_S = 45
INSERT_TIMEOUT_S = 120

COLLECTION_NAME = "Cap3DMM"
DATA_UPLOAD_COLLECTION_NAME = "UploadCap3DMM"


fname = "jeopardy_tiny_with_vectors_all-OpenAI-ada-002.json"  # This file includes pre-generated vectors
url = (
    f"https://raw.githubusercontent.com/weaviate-tutorials/quickstart/main/data/{fname}"
)
resp = requests.get(url)
data = json.loads(resp.text)  # Load data


with weaviate.WeaviateClient(
    connection_params=ConnectionParams.from_params(
        http_host=HTTP_HOST,
        http_port=HTTP_PORT,
        http_secure=HTTP_SECURE,
        grpc_host=GRPC_HOST,
        grpc_port=GRPC_PORT,
        grpc_secure=GRPC_SECURE,
    ),
    additional_config=AdditionalConfig(
        timeout=Timeout(
            init=INTIALISATION_TIMEOUT_S, query=QUERY_TIMEOUT_S, insert=INSERT_TIMEOUT_S
        ),  # Values in seconds
    ),
) as client:
    assert (
        client.is_live()
    ), "Weaviate client is not live"  # This will raise an exception if the client is not live
    print("Client connection established")

    cap3d = client.collections.get(COLLECTION_NAME)
    cap3d_object_count = cap3d.aggregate.over_all(total_count=True).total_count

    print(f"Number of objects in {COLLECTION_NAME} collection: {cap3d_object_count}")

    uuid = "8e88212e-12ce-5942-954f-f49ed469dfe8"  # '71b52390-eb7d-5c9f-8c42-ef60966a0a4f' # '4ce6f46c-d0b0-5b8b-8fe0-c7048eaa09fe'
    obj = cap3d.query.fetch_object_by_id(uuid, include_vector=True)

    if obj:
        print(type(obj.vector["default"]))
        print(len(obj.vector["default"]))
        print(obj.vector["default"][:5])

Client connection established
Number of objects in Cap3DMM collection: 15000
<class 'list'>
512
[0.18559761345386505, 0.09231404215097427, -0.10407475382089615, -0.09906937181949615, 0.07314038276672363]
